# Step 0: Set up directory

In [8]:
# ------------------------------------------------------------
# Download ~10GB of RAW GDELT GKG files from MAY 2018
# Store masterfile + data in separate folders for GitHub structure
# ------------------------------------------------------------

import os
import requests
from tqdm import tqdm
import gzip
import shutil
import zipfile
import shutil

# ------------------------------------------------------------
# WINDOWS PATH YOU REQUESTED
# ------------------------------------------------------------
BASE_DIR = r"C:\Users\jonat\Documents\UMBC\FALL_2025\Final_Project_UMBC_BIG_DATA\DATA"

MASTER_DIR = os.path.join(BASE_DIR, "master")
RAW_DIR = os.path.join(BASE_DIR, "raw")

MASTERFILE = os.path.join(MASTER_DIR, "masterfilelist.txt")
TARGET_GB = 3 # this is for compressed zip files, unzip it's like 3 - 4 times it's size. 

# Ensure folders exist
os.makedirs(MASTER_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)

MASTERLIST_URL = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"


# Step 1: Download masterfilelist.txt → stored in DATA/master

In [9]:
def download_masterfilelist():
    if not os.path.exists(MASTERFILE):
        print("Downloading masterfilelist.txt …")
        r = requests.get(MASTERLIST_URL, timeout=60)
        with open(MASTERFILE, "wb") as f:
            f.write(r.content)
    else:
        print("masterfilelist.txt already exists.")
    return MASTERFILE


master_path = download_masterfilelist()
master_path


masterfilelist.txt already exists.


'C:\\Users\\jonat\\Documents\\UMBC\\FALL_2025\\Final_Project_UMBC_BIG_DATA\\DATA\\master\\masterfilelist.txt'

# Step 2: Collect all May 2018 GKG URLs

In [10]:
def get_may_2018_gkg_urls(path):
    urls = []

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if "gkg.csv.zip" in line:
                parts = line.strip().split()
                url = parts[-1]
                fname = url.split("/")[-1]

                if fname.startswith("201805"):
                    urls.append((fname, url))

    return urls


may_urls = get_may_2018_gkg_urls(master_path)

print(f"Found {len(may_urls)} GKG files for May 2018.")


Found 2853 GKG files for May 2018.


# Step 3: Download raw files until ~10GB, saved in DATA/raw

In [11]:
def download_until_10gb(url_list):
    total_bytes = 0
    target_bytes = TARGET_GB * 1024 * 1024 * 1024

    for fname, url in tqdm(url_list):
        if total_bytes >= target_bytes:
            print(f"\nReached ~{TARGET_GB}GB. Stopping downloads.")
            break

        out_path = os.path.join(RAW_DIR, fname)

        # Skip existing (resume support)
        if os.path.exists(out_path):
            file_size = os.path.getsize(out_path)
            total_bytes += file_size
            continue

        try:
            r = requests.get(url, stream=True, timeout=60)
            if r.status_code != 200:
                print("Failed:", url)
                continue

            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    if chunk:
                        f.write(chunk)

            file_size = os.path.getsize(out_path)
            total_bytes += file_size

            print(f"Downloaded {fname} ({file_size/1024/1024:.2f} MB)")
            print(f"TOTAL: {total_bytes/1024/1024/1024:.2f} GB")

        except Exception as e:
            print("ERROR:", e)
            continue

    print("\nFINAL TOTAL:", total_bytes/1024/1024/1024, "GB")


download_until_10gb(may_urls)


  0%|          | 1/2853 [00:00<24:43,  1.92it/s]

Downloaded 20180501000000.gkg.csv.zip (12.08 MB)
TOTAL: 0.01 GB


  0%|          | 2/2853 [00:01<24:03,  1.97it/s]

Downloaded 20180501001500.gkg.csv.zip (10.73 MB)
TOTAL: 0.02 GB


  0%|          | 3/2853 [00:01<22:04,  2.15it/s]

...

Downloaded 20180503231500.gkg.csv.zip (11.11 MB)
TOTAL: 2.99 GB


 10%|▉         | 272/2853 [02:13<20:41,  2.08it/s]

Downloaded 20180503233000.gkg.csv.zip (11.06 MB)
TOTAL: 3.00 GB


 10%|▉         | 273/2853 [02:14<21:09,  2.03it/s]

Downloaded 20180503234500.gkg.csv.zip (11.68 MB)
TOTAL: 3.01 GB

Reached ~3GB. Stopping downloads.

FINAL TOTAL: 3.0097222616896033 GB


In [12]:


RAW_DIR = r"C:\Users\jonat\Documents\UMBC\FALL_2025\Final_Project_UMBC_BIG_DATA\DATA\raw"
STREAM_IN_DIR = r"C:\Users\jonat\Documents\UMBC\FALL_2025\Final_Project_UMBC_BIG_DATA\DATA\stream_in"

os.makedirs(STREAM_IN_DIR, exist_ok=True)

for fname in os.listdir(RAW_DIR):
    if not fname.endswith(".gkg.csv.zip"):
        continue

    zip_path = os.path.join(RAW_DIR, fname)
    out_name = fname.replace(".zip", "")  # e.g. 20180501000000.gkg.csv
    out_path = os.path.join(STREAM_IN_DIR, out_name)

    if os.path.exists(out_path):
        print(f"Already decompressed, skipping: {out_name}")
        continue

    # Peek at first two bytes to detect format
    with open(zip_path, "rb") as f:
        magic = f.read(2)

    print(f"Decompressing {fname} -> {out_name}")

    try:
        if magic == b"\x1f\x8b":  # gzip signature
            with gzip.open(zip_path, "rb") as f_in, open(out_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)

        elif magic == b"PK":  # zip file signature
            with zipfile.ZipFile(zip_path, "r") as zf:
                # GDELT archives normally contain a single CSV member
                inner_name = zf.namelist()[0]
                with zf.open(inner_name) as f_in, open(out_path, "wb") as f_out:
                    shutil.copyfileobj(f_in, f_out)

        else:
            print(f"Unknown compression format for {fname}, magic bytes={magic!r}")
            continue

    except Exception as e:
        print(f"Error decompressing {fname}: {e}")

print("Done decompressing.")


Decompressing 20180501000000.gkg.csv.zip -> 20180501000000.gkg.csv
Decompressing 20180501001500.gkg.csv.zip -> 20180501001500.gkg.csv
Decompressing 20180501003000.gkg.csv.zip -> 20180501003000.gkg.csv
Decompressing 20180501004500.gkg.csv.zip -> 20180501004500.gkg.csv
Decompressing 20180501010000.gkg.csv.zip -> 20180501010000.gkg.csv
Decompressing 20180501011500.gkg.csv.zip -> 20180501011500.gkg.csv
Decompressing 20180501013000.gkg.csv.zip -> 20180501013000.gkg.csv
Decompressing 20180501014500.gkg.csv.zip -> 20180501014500.gkg.csv
Decompressing 20180501020000.gkg.csv.zip -> 20180501020000.gkg.csv
Decompressing 20180501021500.gkg.csv.zip -> 20180501021500.gkg.csv
Decompressing 20180501023000.gkg.csv.zip -> 20180501023000.gkg.csv
Decompressing 20180501024500.gkg.csv.zip -> 20180501024500.gkg.csv
Decompressing 20180501030000.gkg.csv.zip -> 20180501030000.gkg.csv
Decompressing 20180501031500.gkg.csv.zip -> 20180501031500.gkg.csv
Decompressing 20180501033000.gkg.csv.zip -> 20180501033000.gkg